In [31]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import tqdm
import matplotlib.pyplot as plt
import re

root = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(0, os.path.join(root, 'src'))

from models import models_dir

In [32]:
epochs = 1000
trained_models = os.listdir(f'{models_dir}TSOC')
trained_models = [trained_model for trained_model in trained_models if f'epochs={epochs}' in trained_model]
trained_models

['TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_ort=False_seedmat=0_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=50_opt=sgd_

In [33]:
def parse_parameters(model_name):
    model_name = model_name.split('.pth')[0]
    model_name = model_name.split('TSOC-')[1]
    params = model_name.split('_')
    param_dict = {}
    for param in params:
        key, value = param.split('=')
        # Attempt to convert to int, float, or keep as string
        if value.isdigit():
            param_dict[key] = int(value)
        else:
            try:
                param_dict[key] = float(value)
            except ValueError:
                # Special case for booleans
                if value.lower() in ['true', 'false']:
                    param_dict[key] = value.lower() == 'true'
                else:
                    param_dict[key] = value

    return param_dict

## TSOC reconstruction evaluation

### Evaluate models

In [83]:
for model_name in trained_models:
    params = parse_parameters(model_name)
    orthogonal_flag = "--orthogonal" if params['ort'] else ""
    heart_rate = params['hr'].replace('-', ' ')
    command = f"""
        python ../experiments/decoder_evaluation.py \
            --n {params['n']} \
            --m {params['m']} \
            --seed_training {params['seedtrain']} \
            --mode {params['mode']} \
            --seed_matrix {params['seedmat']} \
            --isnr {params['isnr']} \
            --N_train {params['N']} \
            --train_fraction {params['tf']} \
            --N_test 10000 \
            --basis sym6 \
            --fs {params['fs']} \
            --heart_rate {heart_rate} \
            {orthogonal_flag} \
            --source {params['src']} \
            --processes 48 \
            --gpu 3 \
            --min_lr {params['minlr']} \
            --min_delta {params['mind']} \
            --patience {params['p']} \
            --batch_size {params['bs']} \
            --threshold {params['th']} \
            --epochs {params['epochs']} \
            --lr {params['lr']} \
            --optimizer {params['opt']}
        """
    os.system(command)

INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=standard, orthogonal=True, source=best, seed_matrix=52744, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=standard, orthogonal=False, source=random, seed_matrix=0, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_ort=False_seedmat=0_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=30808, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


...

Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.1, opt=sgd, batch_size=50, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=30808, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=0.001, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25
has already been evaluated


INFO:root:Running test with parameters: n=128, m=32, epochs=1000, lr=0.1, opt=sgd, batch_size=50, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=standard, orthogonal=False, source=random, seed_matrix=0, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=0.001, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC
Traceback (most recent call last):
  File "/home/andriyenttsel/dnn-cs/notebooks/../experiments/decoder_evaluation.py", line 191, in <module>
    test(
  File "/home/andriyenttsel/dnn-cs/notebooks/../experiments/decoder_evaluation.py", line 90, in test
    with open(data_path, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/srv/newpenny/dnn-cs/tsoc/data/ecg_N=10000

Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_ort=False_seedmat=0_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


### Aggregate performance results

In [63]:
# save peformance
results_folder = '/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction'
os.listdir(results_folder)


['TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_or

In [78]:
m_list = [32, 48, 64]
mode_list = ['std', 'rak']
ort_list = [True, False]
seed_mat_list = []
seed_train_list = [0, 1, 2, 3, 4]
opt_list = ['adam', 'sgd']
bs_list = [32, 64, 128]


index = pd.MultiIndex.from_product(
    [m_list, mode_list, ort_list, seed_mat_list, seed_train_list, opt_list, bs_list],
      names=['m', 'mode', 'ort', 'seedmat', 'seedtrain', 'opt', 'bs'])

In [79]:
results = pd.DataFrame(index=index, columns=['P', 'TP', 'TPR', 'TNR', 'ACC', 'RSNR'], dtype=np.float64)
# results = pd.DataFrame(columns=['P', 'TP', 'TPR', 'TNR', 'ACC', 'RSNR'], dtype=np.float64)
for model_name in trained_models:
    params = parse_parameters(model_name)
    results_path = os.path.join(results_folder, f"{model_name.split('.pth')[0]}.pkl")
    if os.path.exists(results_path):
        result = pd.read_pickle(results_path)
        results.loc[params['m'], params['mode'], params['ort'], params['seedmat'], params['seedtrain'], params['opt'], params['bs']] = result

In [80]:
results.loc[48].sort_values('RSNR', ascending=False)

P         TP       TPR  \
mode     ort   seedmat seedtrain opt  bs                                   
rakeness True  30808   0         adam 64  35.219447  29.273587  0.834667   
                                 sgd  50  35.217199  29.159999  0.831367   
standard True  52744   0         adam 64  34.463575  28.784733  0.840723   
                                 sgd  50  34.466099  28.725199  0.838870   
         False 0       0         sgd  50  34.746199  28.941499  0.839155   
                                 adam 64  34.750995  28.851015  0.836358   

                                               TNR       ACC       RSNR  
mode     ort   seedmat seedtrain opt  bs                                 
rakeness True  30808   0         adam 64  0.987873  0.944559  32.521255  
                                 sgd  50  0.986779  0.942905  32.425831  
standard True  52744   0         adam 64  0.987531  0.946291  31.010526  
                                 sgd  50  0.986719  0.945216  30.940460  
         False 0       0         sgd  50  0.986920  0.944869  30.528137  
                                 adam 64  0.986727  0.943984  30.452367

In [85]:
import torch
print(torch.__version__)

2.5.1
